In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
import sklearn.cluster as sc
import time

!pip install hilbertcurve
from hilbertcurve.hilbertcurve import HilbertCurve


# paramètres matplotlib à fixer une fois pour toute
plt.rcParams['image.cmap'] = 'gray'          # choix de colormap
plt.rcParams['image.interpolation'] = 'nearest'  # pas d'interpolation pour l'affichage
plt.rcParams['image.origin'] = 'lower'           # origine en bas à gauche

## Préambule : réutiliser du code Python dans un notebook

Vous pouvez réutiliser du code Python mis en place auparavant.

Pour cela, le plus simple est d'écrire les fonctions utiles dans un fichier `outils.py`, et d'exécuter ensuite `import outils as ou`.


Cela sera utile pour :
- simuler $\mathbf{Y} = \mathbf{y}$ sachant $\mathbf{x}$
- estimer les paramètres
- en fin de TP, comparer les résultats entre le cas "chaîne de Markov" et le cas "indépendance".

In [2]:
class Param_chaine:
    def __init__(self):
        """Constructeur : valeurs initiales."""
        self.pi = np.array([0.5,0.5])
        self.sig = np.array([0.4,0.4])
        self.m = np.array([0.,1.])
        self.A = np.array([[0.999,0.001],
                          [0.001,0.999]])
        
    def print(self):
        """Affichage en une ligne des valeurs de Theta."""
        print("Theta : m = ", self.m, "\t| sig = ",self.sig,"\t| pi =", self.pi,"")
        print("        A[0,0] = ", self.A[0,0],"\t| A[0,1] = ", self.A[0,1] )
        print("        A[1,0] = ", self.A[1,0],"\t| A[1,1] = ", self.A[1,1] )
        
    def toarray(self):
        """Retourne les valeurs de Theta sous la forme d'un tableau."""
        return np.array([self.m[0],self.m[1],self.sig[0],self.sig[1],
                         self.pi[0],self.pi[1],self.A[0,0], self.A[0,1], self.A[1,0], self.A[1,1]
                         ])
    
    def fromarray(self,array):
        """Attribue des valeurs dans Theta à partir d'un tableau."""
        self.m[0],self.m[1],self.sig[0],self.sig[1],self.pi[0],self.pi[1] = array[:6]
        self.A = array[6:].reshape(2,2)
        
def image2chain(image):
    """Transforme une image en chaîne."""
    largeur = image.shape[0]
    p=int(np.log2(largeur**2)); # il faut 2**p pixels dans l'image
    n=2 # image 2D
    hilbert_curve = HilbertCurve(p, n)
    px = [hilbert_curve.point_from_distance(distance)[0] for distance in range(2**p)]
    py = [hilbert_curve.point_from_distance(distance)[1] for distance in range(2**p)]

    chaine = image[px,py]
    return chaine

def chain2image(chaine):
    """Transforme une chaîne en image."""
    p=int(np.log2(chaine.size)); # il faut 2**p pixels dans l'image
    n=2 # image 2D
    hilbert_curve = HilbertCurve(p, n)
    px = [hilbert_curve.point_from_distance(distance)[0] for distance in range(2**p)]
    py = [hilbert_curve.point_from_distance(distance)[1] for distance in range(2**p)]

    largeur = int(np.sqrt(chaine.size))
    out = np.zeros(shape=(largeur,largeur))
    out[px,py] = chaine
    return out

In [3]:
def sample_CMC(Theta,N):
    """ Échantillonage d'une chaîne de Markov de longueur N et de paramètre Theta."""

    X = np.zeros(shape=(N)) 
    # Choix de la valeur initiale
    X[0] = np.random.choice((0,1), p=Theta.pi)
    
    # Récursion avec p(w_i | w_i-1)
    for t in range(1,N):
        X_prec = int(X[t-1])
        X[t] = np.random.choice((0,1), p=Theta.A[X_prec,:])
        
    return X

## 1. Introduction 



<font color='orange' size = 4>**Question 1.**</font>

Lancer le code qui permet de simuler une chaîne de Markov, puis l'afficher en 1D et en 2D.
Faire varier les paramètres et commenter.

Pour la taille de l'image, on prendra ici et dans la suite 128 x 128 pixels.

<font color='orange' size = 4>**Question 2.**</font>

Générer une observation $\mathbf{Y} = \mathbf{y}$ à partir d'une réalisation de chaîne de Markov. **Ne plus changer cette image**, c'est celle qui vous servira dans les questions suivantes.

## 2. Segmentaton supervisée

<font color='orange' size = 4>**Question 3.**</font>

Calculer la vraisemblance $p(y_n | x_n = \omega_i)$ pour tout point $n$ et pour toute classe $i$. Afficher le résultat pour vérifier que les calculs sont corrects.

<font color='orange' size = 4>**Question 4.**</font>

Calculer $\alpha(n,i)$ pour tout point $n$ et classe $i$. Le résultat sera contenu dans un tableau `alpha` de taille $(N,2)$.

Afficher le résultat sous la forme de deux courbes. Que constate-t'on dès que $n$ devient trop grand ?

<font color='orange' size = 4>**Question 5.**</font>

On se propose de remédier à ce problème en normalisant les valeurs de $\alpha$ au fur et à mesure du calcul. Cette normalisation sera faite de telle sorte que $\alpha(n,0) + \alpha(n,1) = 1$ pour tout $n$.

Afficher les nouvelles valeurs de $\alpha$ pour vérifier que le problème est résolu.

<font color='orange' size = 4>**Question 6.**</font>

Calculer maintenant $\beta(n,i) \forall n, \forall i$. Comme le problème précédent existe aussi pour le calcul des $\beta$, il est possible d'y remédier en effectuant là aussi une normalisation. 

<font color='orange' size = 4>**Question 7.**</font>

À partir des $\alpha, \beta$ calculés précédemment, calculer les probabilités à posteriori $\xi(n,i)$ puis les afficher.

En déduire la segmentation de $\mathbf{x}$ au sens du MPM, afficher le résultat sous forme d'image, et indiquer le taux d'erreur.

## 3. Segmentation non supervisée.

Nous allons ici implémenter l'algorithme SEM. Pour cela il est nécessaire de savoir :
- Estimer $\mathbf{\Theta}$ à l'aide de données complètes $(\mathbf{x}, \mathbf{y})$
- Simuler une réalisation de chaîne de Markov *a posteriori*, et donc : calculer les probabilités de transition a posteriori

<font color='orange' size = 4>**Question 8.**</font>

Écrire une fonction `est_A` qui estime la matrice des probabilité de transition d'une classe vers l'autre dans une chaîne de Markov `x`.

Reprendre les estimateurs du maximum de vraisemblance déjà réalisés pour la moyenne $\mu$, l'écart type $\sigma$, et les probabilités d'apparition $\pi$.

Rassembler tous les estimateurs dans une fonction `est_Theta_chaine` qui estime l'ensemble des paramètres du modèle à partir d'un couple $\mathbf{x}, \mathbf{y}$.

<font color='orange' size = 4>**Question 9.**</font>

Calculer :
- $\psi(i,j,n)$ à partir de $\alpha$, $\beta$ et $\xi$
- tous les $p(x_{n+1}=\omega_j | x_n=\omega_i, \mathbf{y})$ avec $\xi(i,j,n)$ et $\psi(i,j,n)$

Rassembler ces calculs dans une fonction `calc_ptrans_post`. 

Pour vérifier que les calculs sont corrects, on suppose $\Theta$ connu (on prendra par exemple `Theta_reel`) : simuler une réalisation *a posteriori* de $\mathbf{x}$.

<font color='orange' size = 4>**Question 10.**</font>

Implémenter l'algorithme SEM. L'on pourra suivre les étapes vues dans le cas indépendant :

1. Initialiser simplement en prenant un estimateur naïf de $\mathbf{x}$
2. Fixer un grand nombre d'itération ( $> 100$ au moins)
3. Vérifier que la valeur finale de `Theta_SEM` est raisonnable par rapport à `Theta_reel` défini plus haut.

Ensuite, nous pouvons être plus précis algorithmiquement :

4. Stocker chaque valeur de $\mathbf{\Theta}^{(q)}$ dans une ligne d'un tableau `theta_series` de taille adéquate.
5. Afficher toutes les valeurs prises par les paramètres en affichant 6 graphes d'évolution des valeurs en fonction du nméro d'itération.
6. Prendre comme valeur finale de `Theta_SEM` la moyenne des dernières valeurs.

<font color='orange' size = 4>**Question 11.**</font>

Finalement, réaliser une segmentation non supervisée de l'image $\mathbf{y}$ réalisée en question 2, et calculer le taux d'erreur. Le comparer à :
- Celui obtenu dans le cas supervisé (question 7)
- Celui obtenu dans le cas indépendant

# 4. Variations expérimentales

<font color='orange' size = 4>**Question 12.**</font>

Faire varier la valeur prise par $\sigma$, et afficher les courbes de résultats moyens, pour le cas supervisé et non supervisé, avec une image $\mathbf{y}$ réalisée à partir d'une chaîne de Markov. Commenter.

<font color='orange' size = 4>**Question 13.**</font>

Faire de même avec l'image `cat128.png` disponible sur Moodle, et commenter.